In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
forest_cover = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')
forest_cover.head()

There is a lot of examples with type 2 and 1 but few other types

In [ ]:
forest_cover['Cover_Type'].value_counts().plot(kind='bar')

The Dataframe have 0 null elemnts

In [ ]:
forest_cover.isna().sum()

In [ ]:
forest_cover.describe()

In [ ]:
forest_cover.drop(['Id'], axis=1, inplace=True)
forest_cover.head()

Get test dataframe

In [ ]:
test_cover = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')
test_cover.drop(['Id'], axis=1, inplace=True)
test_cover.head()

## Feature Engineer

**Aspect** 

Aspect is the compass direction that a terrain faces. Here it is expressed in Sexagesimal system where the angle lies in the range (0, 359) degrees. In this feature, however, there are some values which are less than 0 and some values which are greater than 359. It will be better If we fix those values so that It lies in the given range. This is fairly easy to do in this case because upon a closer inspection you will find that all the values in this column lies in the range (-360, 720). So, adding 360 to angles smaller than 0 and subtracting 360 from angles greater than 359 will do the work.

Discussion thread: https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/293373

In [ ]:
forest_cover['Aspect'].value_counts().sort_values(ascending=False)

Fixing Aspect 

In [ ]:
forest_cover["Aspect"][forest_cover["Aspect"] < 0] += 360
forest_cover["Aspect"][forest_cover["Aspect"] > 359] -= 360

test_cover["Aspect"][test_cover["Aspect"] < 0] += 360
test_cover["Aspect"][test_cover["Aspect"] > 359] -= 360

In [ ]:
forest_cover['Aspect'].value_counts().sort_values(ascending=False)

# Separate features and target

In [ ]:
y = forest_cover['Cover_Type']
X = forest_cover.drop(['Cover_Type'], axis=1)

In [ ]:
y.head()

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X_scaler = scaler.fit_transform(X)
X_scaler[:2]

In [ ]:
len(X_scaler[1])

In [ ]:
test_scaler = scaler.transform(test_cover)
test_scaler[:2]

In [ ]:
del forest_cover
del X
del test_cover

# Transform target to array

In [ ]:
# transformar labels a 1 y 0 
from tensorflow.keras.utils import to_categorical

In [ ]:
list(y)

In [ ]:
y_cat = to_categorical(list(y))

In [ ]:
y_cat[0]

## Create train and valid sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_val, y_train, y_val = train_test_split(X_scaler, y_cat, test_size=0.30, random_state=42)

In [ ]:
del X_scaler
del y
del y_cat

# Create a model with Keras

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu' , input_shape=[54]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(8, activation='softmax')
])

In [ ]:
early_stopping = callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
y_train[:4]

In [ ]:
y_val[:4]

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=512,
    epochs=100
)

In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
history_df.loc[5:, ['loss', 'val_loss']].plot()

0.9481 V2


- 1 - 5469/5469 [==============================] - 87s 16ms/step - loss: 0.1068 - accuracy: 0.9537 - val_loss: 0.0893 - val_accuracy: 0.9594

- 3 - 5469/5469 [==============================] - 68s 12ms/step - loss: 0.1270 - accuracy: 0.9478 - val_loss: 0.1001 - val_accuracy: 0.9554

## Make a prediction

In [ ]:
predict = model.predict(test_scaler)

In [ ]:
predict[:10]

In [ ]:
from numpy import argmax

In [ ]:
def create_result(matrix):
    predict = []
    
    for m in matrix:
        predict.append(argmax(m))
        
    return predict

In [ ]:
test_result = create_result(predict)
test_result[:10]

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')
submission.head(10)

In [ ]:
submission['Cover_Type'] = test_result
submission.head(10)

In [ ]:
submission.to_csv('submission.csv',index=False)